# Creating a MOCK dataset to make classification and recommendation
Author: Karina Condeixa

CONDITION:
 "available": true

QUESTIONS
- number for views 
- like (with % of views-maximum?)

TODO:
- item_gone_timer = datetime_post - datetime_with_status_available
- item_lat_lng
- check lat_lng_samples.csv


In [21]:
# import packages
import pandas as pd
from faker import Faker, providers
from faker.providers.address.de_DE import Provider as DeDeAddressProvider
from faker.generator import random
from faker.providers import BaseProvider
# import random

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# %matplotlib notebook
# %matplotlib 

import folium
# from geopy.geocoders import Nominatim

In [22]:
# define global variables

maximun_n_views = 2001
pct_of_likes_over_views = 20
n_records_ranking = 5000
n_records = 5000

limit = '-30d'  # limit of 30 days of item in the app
maximun_n_hours_avalilable = 100

random_seed = 1000


# post_date: something between the last 30 days - limit = '-30d'
# timer = (date.today()) - post_date

In [23]:
category = ['furniture',  'clothes',  'kitchen_utensils', 'kids', 'other'] 
condition = ['poor', 'acceptable', 'good', 'like_new'] 
available = ['1', '0']    

# postcodes = postcodes_berlin_series
    
n_views = []
for v in range(maximun_n_views):  # until 200 views
    n_views.append(v)

pct_likes = []
for l in range(pct_of_likes_over_views):  # until 20% of the n_views
    pct_likes.append(l/100)
 
# datetime post - datetime now, I create a maximum number of hours
n_hours_avalilable = []
for h in range(maximun_n_hours_avalilable):  
    n_hours_avalilable.append(h)
    
distance = []
for d in range(32000):
    distance.append(d)
    


### Postcodes

In [24]:
# immport and clean original dataset removing poostcodes from Germanay, that are not in Berlin
postcodes_de = pd.read_excel(r'data/German-Zip-Codes.xlsx', sheet_name='Berlin')
df = pd.DataFrame(postcodes_de)
df.set_axis(["postcodes_berlin"], axis=1,inplace=True)
df = (df["postcodes_berlin"].str[8:-11])
df.to_csv('data/postcodes_berlin.csv', index=False)

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_18092/2575223029.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(["postcodes_berlin"], axis=1,inplace=True)


### Creating the postcode series

In [25]:
# import postcodes from Berlin and create a dataframe removing indexes and headers
postcodes_berlin = pd.read_csv('data/postcodes_berlin.csv')
print(postcodes_berlin)
postcodes_berlin_series = postcodes_berlin[:][1:].squeeze()

     postcodes_berlin
0               10117
1               10119
2               10178
3               10179
4               10243
..                ...
184             14169
185             14193
186             14195
187             14197
188             14199

[189 rows x 1 columns]


### Creating lists

In [26]:
# Importing Modules
import numpy as np
import random
# Use 'conda install shapely' to import the shapely library.
from shapely.geometry import Polygon, Point



### Creating the datasets

In [27]:
# Note: # multi_locale_generator = Faker(['it_IT', 'en_US', 'de-DE', 'pt_BR', 'es-ES', 'fr-FR', 'ru-RU', 'tr-TR'])

# Instantiate Faker with multiple locales
german_locale_generator = Faker(['de_DE'])
fake = Faker()
Faker.seed(0)


### List of latitude and longitude co-ordinates 
Based on a sample of co-ordinates from [free data Berlin](https://daten.berlin.de/datensaetze/stra%C3%9Fenverkehrsunf%C3%A4lle-nach-unfallort-berlin-2021) for Road traffic accidents by accident location in Berlin 2021.


In [28]:
lat_lng_samples = pd.read_csv("data/lat_lng_samples.csv")
lat_lng_samples.head(2)

,lat,lng
0,"5,253,393,955","1,342,689,483"
1,"5,243,995,086","1,339,209,027"


In [29]:
# replace comma to dot
lat_lng_samples = lat_lng_samples.apply(lambda x: x.str.replace(',','.'))

In [30]:
list_lat = lat_lng_samples.lat.values.tolist()
list_lng = lat_lng_samples.lng.values.tolist()
lat_lng_samples.head(2)


,lat,lng
0,5.253.393.955,1.342.689.483
1,5.243.995.086,1.339.209.027


In [31]:
lat_lng_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11197 entries, 0 to 11196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     11192 non-null  object
 1   lng     11192 non-null  object
dtypes: object(2)
memory usage: 175.1+ KB


### Item dataset

In [32]:
# An initial classification that is not being used to normalize with UXs categories. But it is saved for future updates.
# category_item = ['furniture-sofa',
#                  'furniture-armchair',
#                  'furniture-chair',
#                  'furniture-table',
#                  'furniture-desk',
#                  'furniture-bed',
#                  'furniture-bookcase',
#                  'furniture-bedside_table',
#                  'furniture-cabinet',
#                  'furniture-wardrobe',
#                  'furniture-shelf',
#                  'furniture-cupboard',
#                  'furniture-rollcontainers',
#                  'furniture-shoe_rack',
#                  'furniture-mirror',
#                  'furniture-cot',
#                  'furniture-trolley',
#                  'appliance-washing_machine',
#                  'appliance-dish_washer',
#                  'appliance-drying_rack',
#                  'appliance-refrigerator',
#                  'appliance-blender',
#                  'appliance-extractor_hood',
#                  'appliance-clothes_iron',
#                  'appliance-vacuum_cleaner',
#                  'appliance-sandwich_maker',
#                  'appliance-kettle',
#                  'appliance-air_conditioner',
#                  'appliance-heater',
#                  'appliance-pan',
#                  'appliance-popcorn_maker',
#                  'appliance-coffee_machine',
#                  'appliance-stove',
#                  'lighting-lighting',
#                  'lighting-chandelier',
#                  'lighting-lightbulb',
#                  'musical_equipment-guitar',
#                  'musical_equipment-sound_amplifier',
#                  'musical_equipment-contrabass',
#                  'musical_equipment-battery',
#                  'musical_equipment-piano',
#                  'tech-desktop',
#                  'tech-laptop',
#                  'tech-phone',
#                  'tech-keyboard',
#                  'clothes-woman_jacket',
#                  'clothes-man_jacket',
#                  'clothes-child_jacket',
#                  'clothes-woman_clothes',
#                  'clothes-man_clothes',
#                  'clothes-child_clothes',
#                  'shoes-woman_shoes',
#                  'shoes-man_shoes',
#                  'shoes-child_shoes',
#                  'miscelaneaous-ironing_board',
#                  'miscelaneaous-picture_frame',
#                  'miscelaneaous-bicycle',
#                  'miscelaneaous-plant',
#                  'miscelaneaous-carpet',
#                  'miscelaneaous-roller_skates',
#                  'miscelaneaous-ski_skates',
#                  'miscelaneaous-books',
#                  'miscelaneaous-purse',
#                  'miscelaneaous-suitcase',
#                  'miscelaneaous-shopping_venture',
#                  'miscelaneaous-board',
#                  'miscelaneaous-frame',
#                  'home-mattress', 
#                  'home-carpet',
#                  'kids-stroller',
#                  'kids-baby_carriage']

In [33]:
# addresses = []
# for _ in range(50000000):
#     address = german_locale_generator.address()
#     if 'Berlin' in address:
#         addresses.append(address)
#     else:
#         if len(addresses) < 1500: 
#             continue
#         else: 
#             break
            
# df['is_berlin'] = df['address'].apply(lambda x: 1 if 'Berlin' in x else 0)

In [34]:

#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

#         item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')

In [35]:
# # Define the desired polygon : points choosen in Google maps

# poly = Polygon([(52.645883, 13.395869), 
#                 (52.526568, 13.645808),
#                 (52.381789, 13.405482),
#                 (52.484773, 13.136317)])


# min_x = 52.381789
# max_x = 52.645883
# min_y = 13.136317
# max_y = 13.645808

# # Defining the randomization generator
# def polygon_random_points (poly, num_records):
#     min_x, min_y, max_x, max_y = poly.bounds
#     points = []
#     while len(points) < num_records:
#         random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
#         if (random_point.within(poly)):
#             points.append(random_point)
#     return points
    # Choose the number of points desired. T\ 
# points = polygon_random_points(poly,n_records)
# # Testing the results.
# for p in points:
#     print(p.x,",",p.y)


In [36]:
# from math import sin, cos, sqrt, atan2, radians

# # Approximate radius of earth in km
# R = 6373.0

# lat1 = radians(52.2296756)
# lon1 = radians(21.0122287)
# lat2 = radians(52.406374)
# lon2 = radians(16.9251681)

# dlon = lon2 - lon1
# dlat = lat2 - lat1

# a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
# c = 2 * atan2(sqrt(a), sqrt(1 - a))

# distance = R * c

# print("Result: ", distance)
# print("Should be: ", 278.546, "km")

In [40]:
from datetime import date

# random.seed(random_seed)

# points = polygon_random_points(poly,n_records)

def create_data_classification_1(num_records): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records): 
        item[i] = {}
#         item[i]['item_status'] = 1
        item[i]['item_category'] = np.random.choice(category)
        item[i]['item_condition'] = np.random.choice(condition)

#         item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable) # in 30 days
        
#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

        item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
    
        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')  # in days, can be improved to hours
        item[i]['distance'] = np.random.choice(distance)
        
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
        
        item[i]['item_available'] = np.random.choice(available)
             
    return item

In [41]:
ml_class_df = pd.DataFrame(create_data_classification_1(n_records)).transpose()
ml_class_df.head()

,item_category,item_condition,item_postcode,item_available_timer,distance,n_views,n_likes,item_available
0,other,good,10781,64,7001,1460,116,1
1,kitchen_utensils,good,14053,18,1668,1399,69,0
2,kitchen_utensils,poor,13189,24,1396,441,8,0
3,kitchen_utensils,good,12437,30,28835,1162,220,0
4,kids,like_new,13591,53,11339,1652,115,1


In [42]:
ml_class_df['item_id'] = ml_class_df.index +1  # add item_id
item_id_series = ml_class_df['item_id']  # storage in a variable to use later

ml_class_df.head(10)

,item_category,item_condition,item_postcode,item_available_timer,distance,n_views,n_likes,item_available,item_id
0,other,good,10781,64,7001,1460,116,1,1
1,kitchen_utensils,good,14053,18,1668,1399,69,0,2
2,kitchen_utensils,poor,13189,24,1396,441,8,0,3
3,kitchen_utensils,good,12437,30,28835,1162,220,0,4
4,kids,like_new,13591,53,11339,1652,115,1,5
5,other,acceptable,12101,78,3281,692,83,0,6
6,clothes,like_new,10623,84,23699,1174,211,0,7
7,furniture,like_new,12587,98,24175,579,110,1,8
8,kids,good,12101,52,16601,1218,194,0,9
9,clothes,like_new,13583,76,13889,65,4,1,10


### User

In [ ]:
# user_df = item_df.copy()

In [ ]:
# user_df = user_df.rename(columns={"item_id": "user_id"})

In [ ]:
# user_df.head(10)

In [ ]:
# # item_df.get('item_timer')

# item_df['item_timer'] = item_df['item_timer'].astype(str).str[:2].str.strip()
# item_df['item_timer'] = item_df['item_timer'].astype(int)

# item_df = item_df.rename(columns={'item_timer': 'item_timer_days'})

# # item_df.drop('item_timer', axis=1)
# item_df.info()

# item_df.head()

In [ ]:
# # saves csv files
# ml_class_df.to_csv('data/ml_class_df.csv', index=False)
# # user_df.to_csv('data/user_df.csv', index=False)

In [43]:



def create_data_rankings(num_records_ranking): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records_ranking): 
        item[i] = {}
#         item[i]['item_status'] = 1
        item[i]['item_category'] = np.random.choice(category)
        item[i]['item_condition'] = np.random.choice(condition)

        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now') # in days, can be improved to 
        
        item[i]['distance'] = np.random.choice(distance)
        
        
 
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
        
             
    return item

In [44]:
ranking_df = pd.DataFrame(create_data_rankings(n_records_ranking)).transpose()
ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes
0,clothes,good,20,26676,135,25
1,kitchen_utensils,acceptable,17,10486,1023,153
2,other,poor,86,9973,723,57
3,other,acceptable,34,5810,1901,57
4,other,like_new,32,12589,393,47


In [45]:
ranking_df['item_id'] = ranking_df.index +1  # add item_id
item_id_series = ranking_df['item_id']  # storage in a variable to use later

ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_id
0,clothes,good,20,26676,135,25,1
1,kitchen_utensils,acceptable,17,10486,1023,153,2
2,other,poor,86,9973,723,57,3
3,other,acceptable,34,5810,1901,57,4
4,other,like_new,32,12589,393,47,5


In [46]:
# ranking_df['item_condition']
# ['poor', 'acceptable', 'Good', 'like_new'] 

ranking_df['item_condition'] = ranking_df.item_condition.replace(['like_new', 'good', 'acceptable', 'poor'], [1, 2, 3, 4])

ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_id
0,clothes,2,20,26676,135,25,1
1,kitchen_utensils,3,17,10486,1023,153,2
2,other,4,86,9973,723,57,3
3,other,3,34,5810,1901,57,4
4,other,1,32,12589,393,47,5


In [47]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_category         5000 non-null   object
 1   item_condition        5000 non-null   int64 
 2   item_available_timer  5000 non-null   object
 3   distance              5000 non-null   object
 4   n_views               5000 non-null   object
 5   n_likes               5000 non-null   object
 6   item_id               5000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 312.5+ KB


In [48]:
# saves csv files
ranking_df.to_csv('../recommendation/data/ranking_df.csv', index=False)

In [49]:
ml_class_df.to_csv('data/ml_class_df.csv', index=False)